In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-white')
plt.rc('figure', dpi=100, figsize=(7, 5))
plt.rc('font', size=12)

# Lecture 22 – Modeling and `sklearn`

## DSC 80, Spring 2022

### Announcements

- Discussion 8 (regression) is today from 7-8:30PM.
- Project 4 has been released!
    - The checkpoint is due **tomorrow at 11:59PM**.
    - The full project is due **Thursday, May 26th at 11:59PM**.
    - Start early!
- Lab 8 is due on **Monday, May 23rd at 11:59PM**.

### Agenda

- Example: Restaurant tips 🧑‍🍳.
- `sklearn` overview.
- Transformers in `sklearn`.
- Models in `sklearn`.

## Example: Restaurant tips 🧑‍🍳

In [ ]:
tips = sns.load_dataset('tips')
tips

### Model #1: Constant

The first model we looked at last class used a constant tip **amount** prediction for every table.

$$\text{predicted tip} = h^*$$

If we use squared loss, the "best prediction is the mean of the observed tips.

In [ ]:
mean_tip = tips['tip'].mean()
mean_tip

In [ ]:
def rmse(actual, pred):
    return np.sqrt(np.mean((actual - pred) ** 2))

In [ ]:
rmse(tips['tip'], mean_tip)

In [ ]:
rmse_dict = {}
rmse_dict['constant, tip'] = rmse(tips['tip'], mean_tip)

### Model #2: Tip percentages instead of tips

The next model we created used a constant tip **percentage** prediction for every table.

$$\text{predicted tip percentage} = h^*$$

In [ ]:
tips = tips.assign(pct_tip=(tips['tip'] / tips['total_bill']))
tips.head()

In [ ]:
mean_pct_tip = tips['pct_tip'].mean()
mean_pct_tip

- Computing the RMSE of this model is a bit more nuanced.
- To fairly compare this model to the previous model, we must still be predicting `'tip'`, but above we have predicted `'pct_tip'`.
- **Key idea:** `'pct_tip'` is a **multiplier** that we apply to `'total_bill'` to get `'tip'`. That is:

$$\text{predicted tip} = \text{total bill} \cdot \text{mean pct-tip}$$

In [ ]:
tips['total_bill'] * mean_pct_tip

In [ ]:
rmse_dict['constant, pct_tip'] = rmse(tips['tip'], tips['total_bill'] * mean_pct_tip)
rmse_dict

### Constant tip vs. constant tip percentage

In [ ]:
mean_pct_tip

In [ ]:
rmse_dict

- A constant prediction of 16.08\% yields a lower RMSE than a constant prediction of \\$3.
- However, both RMSEs are over \\$1, which is relatively high compared to the mean tip amount of \\$3.
- How can we bring this RMSE down?

### Model #3: Linear model

* **Model:** Tips are made according to a linear function:

$$\text{predicted tip} = w_0^* + w_1^* \cdot \text{tip}$$

- By choosing a loss function and minimizing empirical risk, we can find $w_0^*$ and $w_1^*$.
    - This process is **fitting** our model to the data.
    - $w_0^*$ and $w_1^*$ can be thought of as estimates of the true intercept and slope that exist in nature.
    
- In order to use a linear model, the data should have a linear association.

In [ ]:
sns.lmplot(data=tips, x='total_bill', y='tip', line_kws={'color': 'red'});

### Fitting a linear model

We'll learn more about `sklearn` today.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(X=tips[['total_bill']], y=tips['tip'])

In [ ]:
lr.intercept_, lr.coef_

Note that the above coefficients state that the "best way" (according to squared loss) to make tip predictions using a linear model is to assume people
- Tip ~\\$0.92 up front, and
- ~10.5\% of every dollar thereafter.

In [ ]:
preds = lr.predict(X=tips[['total_bill']])
rmse_dict['simple linear model'] = rmse(tips['tip'], preds)
rmse_dict

### Recap

- We built three models:
    - A constant model: $\text{predicted tip} = h^*$.
    - A linear model with no intercept: $\text{predicted tip} = w^* \cdot \text{total bill}$.
        - This was the model that involved tip percentage.
    - A linear model with an intercept: $\text{predicted tip} = w_0^* + w_1^* \cdot \text{total bill}$.
- As we added more features, our RMSEs decreased.
    - This was guaranteed to happen, since we were only looking at our training data.
- It is not clear that the final linear model is actually "better"; it doesn't seem to **reflect reality** better than the previous models.

### What's next?

There's a lot of information in `tips` that we didn't use – `'sex'`, `'smoker'`, `'day'`, and `'time'`, for example. How might we **encode** this information?

In [ ]:
tips.head()

### One-hot encoding categorical variables

- Recall, we use **one-hot encoding** to transform **nominal categorical columns** into **binary columns**.
- Are all of the categorical columns in `tips` nominal?
    - Answer: `'day'` is ordinal, but we _can_ still use one-hot encoding.

In [ ]:
tips.head()

In [ ]:
categorical_cols = ['sex', 'smoker', 'day', 'time']

Let's one-hot encode the categorical columns in `tips`. Here's another way to do so manually:

In [ ]:
features = tips.copy().loc[:, ['total_bill', 'size']]
for c in categorical_cols:
    for val in tips[c].unique():
        features[f'{c}={val}'] = (tips[c] == val).astype(int)

In [ ]:
features.head()

Note that `features` has the same number of rows as `tips`; it just has more columns.

In [ ]:
tips.shape

In [ ]:
features.shape

Let's fit a linear model using all features in `features`!

In [ ]:
lr = LinearRegression()
lr.fit(X=features, y=tips['tip'])

In [ ]:
rmse_dict['all features'] = rmse(tips['tip'], lr.predict(X=features))
rmse_dict

The RMSE of our latest model is the lowest of all linear models we've built so far (which is to be expected), **but not by much**. Perhaps these latest features weren't that useful.

We can visualize our latest model's predictions, too:

In [ ]:
preds = lr.predict(X=features)

plt.figure(figsize=(10, 5))
plt.scatter(tips['total_bill'], tips['tip'], label='actual tips')
plt.scatter(tips['total_bill'], preds, label='predicted tips')
plt.xlabel('total_bill')
plt.ylabel('tip')
plt.legend();

Why don't our model's predictions lie on a straight line? 🤔

### Aside: Periodic sales

- Recall, we can transform existing quantitative features into new quantitative features.
    - Often, we do this to make **linear relationships** more evident.
- Here, we have a dataset consisting of daily sales volumes from a (fictional) store.
- We'd like to **predict future sales**, given current trends.
    - What is the "shape" of the relationship between `'day'` and `'units sold'`?
        - Why is there periodicity ("peaks" and "valleys")?
    - How can we **transform** this relationship so that a linear model is suitable?

In [ ]:
sales = pd.read_csv('data/sinusoidal.csv').sort_values(by='day').reset_index(drop=True)
sales.plot(kind='scatter', x='day', y='units sold', title='Daily Sales Volume');

In [ ]:
sns.lmplot(data=sales, x='day', y='units sold', line_kws={'color': 'red'});
sns.residplot(data=sales, x='day', y='units sold', color='orange', label='residuals');
plt.title('Units Sold vs. Day')
plt.legend();

There is a **pattern** in the residual plot here, which is indicative that a linear model is not the best choice.

### Example: Periodic sales

- **Solution:** Transform either `'day'` or `'units_sold'` so that the resulting relationship between the two variables is roughly linear.
- Observation: It appears that sales are **sinusoidal**.
    - The "peaks" are spaced out by 7 days (1 week), so the period of the sinusoid is 7.
    - The height difference between a peak and a valley is approximately 10, so the amplitude of the sinusoid is $\frac{10}{2} = 5$.
    - Sales seem to be made up of a linear growth term and a sinusoidal term.
- As such, we can **transform `'day'`** using the feature function:

$$ \phi(x) = x + 5\sin\left(\frac{2\pi}{7} \cdot x\right) $$

In [ ]:
def transform_day(day):
    return day +  5 * np.sin(2 * np.pi * day / 7)

In [ ]:
sales['day_transformed'] = transform_day(sales['day'])

Let's draw two scatter plots:
- `'day_transformed'` vs. `'day'`.
- `'units sold'` vs. `'day'`.

In [ ]:
plt.scatter(sales['day'], sales['day_transformed'], label='day_transformed')
plt.scatter(sales['day'], sales['units sold'], label='units sold')
plt.xlabel('day')
plt.legend();

While neither the orange scatter plot nor the blue scatter plot look linear, the relationship between the $y$-values in the two scatter plots _is_ roughly linear!

Our new linear model will use `'day_transformed'` as the $x$ and `'units sold'` as the $y$.

In [ ]:
sales.plot(kind='scatter', x='day_transformed', y='units sold');

In [ ]:
sns.lmplot(data=sales, x='day_transformed', y='units sold', line_kws={'color': 'red'})
sns.residplot(data=sales, x='day_transformed', y='units sold', color='orange', label='residuals')
plt.title('Units Sold vs. Transformed Day')
plt.legend();

Now, the residual plot seems random, which is ideal!

## `sklearn` overview

### The steps of the modeling pipeline

<center><img src="imgs/image_0.png" width="60%"></center>

1. Create features to best reflect the "meaning" behind data.
2. Choose a model that is appropriate to capture the relationships between features and the response.
3. Select a loss function and fit the model (i.e., determine $w^*$).
4. Evaluate the model (e.g. using RMSE).

### Features and models using `sklearn`

<center><img src="imgs/sklearn.png" width="20%"></center>
    
* Scikit-learn (`sklearn`) implements many common steps in the feature and model creation pipeline.
    - It is **widely** used throughout [industry](https://scikit-learn.org/stable/testimonials/testimonials.html#:~:text=It%20is%20very%20widely%20used,very%20approachable%20and%20very%20powerful.) and academia.
* It interfaces with `numpy` arrays, and to an extent, `pandas` DataFrames.
* Huge benefit: the [documentation online](https://scikit-learn.org/stable/modules/classes.html) is **excellent**.

### `preprocessing` and `linear_models`

For the **feature creation** step of the modeling pipeline, we will use `sklearn`'s [`preprocessing`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) module.

<center><img src="imgs/feature_part.png" width="30%"></center>

For the **model creation** step of the modeling pipeline, we will use `sklearn`'s [`linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) module.

<center><img src="imgs/model_part.png" width="36%"></center>

## Transformers in `sklearn`

### Transformer classes

- **Transformers** take in "raw" data and output "processed" data. They are used for **creating features**.
    - The input should be a multi-dimensional `numpy` array.
        - Inputs can be DataFrames, but `sklearn` only looks at the values (i.e. it calls `to_numpy()` on input DataFrames).
    - The output is a `numpy` array (never a DataFrame or Series).

- Transformers, like most relevant features of `sklearn`, are **classes**, not functions, meaning you need to instantiate them and call their methods.

### Example transformer: `Binarizer`

The `Binarizer` transformer allows us to map a quantitative sequence to a sequence of 1s and 0s, depending on whether values are above or below a threshold.

|Property|Example|Description|
|---|---|---|
|Initialize with parameters| `binar = Binarizer(thresh)` | set x=1 if x > thresh, else 0|
|Transform data in a dataset | `feat = binar.transform(data)` | Binarize all columns in `data`|

First, we need to import the relevant class from `sklearn.preprocessing`. (Tip: import just the relevant classes you need from `sklearn`.)

In [ ]:
from sklearn.preprocessing import Binarizer

Let's try binarizing `'total_bill'`. We'll say a "large" bill is one that is over \$20.

In [ ]:
tips = sns.load_dataset('tips') # To remove the columns we "engineered" before
tips['total_bill'].head()

First, we initialize a `Binarizer` object with the threshold we want.

In [ ]:
bi = Binarizer(threshold=20)

Then, we call `bi`'s `transform` method and pass it the data we'd like to transform. Note that its input and output are both 2D.

In [ ]:
transformed_bills = bi.transform(tips[['total_bill']]) # Must pass transform a 2D array/DataFrame
transformed_bills[:5]

Cool! We can verify that it worked correctly:

In [ ]:
((tips['total_bill'] > 20).astype(int) == transformed_bills.flatten()).all()

### Example transformer: `StdScaler`

- `StdScaler` **standardizes** data using the mean and standard deviation of the data.

$$z_i = \frac{x_i - \bar{x}}{\sigma_x}$$

- Unlike `Binarizer`, `StdScaler` **requires some knowledge (mean and SD) of the dataset before transforming**.
- As such, we need to **`fit`** an `StdScaler` transformer before we can use the `transform` method.
* Typical usage: fit transformer on a sample; use that fit transformer to transform future data.


|Property|Example|Description|
|---|---|---|
|Initialize with parameters| `stdscaler = StandardScaler()` | z-scale the data (no parameters) |
|Fit the transformer| `stdscaler.fit(data)` | compute the mean and SD of `data`|
|Transform data in a dataset | `feat = stdscaler.transform(newdata)` | z-scale `newdata` with mean and SD of `data`|

It only makes sense to standardize the already-quantitative columns of `tips`, so let's select just those.

In [ ]:
tips_quant = tips[['total_bill', 'tip', 'size']]
tips_quant.head()

Let's initialize a `StandardScaler` object.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
stdscaler = StandardScaler()

Note that the following **does not work!** The error message is very helpful.

In [ ]:
stdscaler.transform(tips_quant)

Instead, we need to first call the `fit` method on `stdscaler`.

In [ ]:
stdscaler.fit(tips_quant)

Now, `transform` will work.

In [ ]:
# First column is 'total_bill', second column is 'tip', third column is 'size'
tips_quant_z = stdscaler.transform(tips_quant)
tips_quant_z[:5]

We can also access the mean and variance `stdscaler` computed for each column:

In [ ]:
stdscaler.mean_

In [ ]:
stdscaler.var_

Note that we can call `transform` on DataFrames other than `tips_quant`:

In [ ]:
stdscaler.transform(tips_quant.head(5))

### Example transformer: `OneHotEncoder`

Let's keep just the categorical columns in `tips`.

In [ ]:
tips_cat = tips[['sex', 'smoker', 'day', 'time']]
tips_cat.head()

Like `StdScaler`, we will need to `fit` our `OneHotEncoder` transformer before it can transform anything.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder()
ohe.fit(tips_cat)

We can look at the unique values (i.e. categories) in each column by using the `categories_` attribute:

In [ ]:
ohe.categories_

In [ ]:
ohe_features = ohe.transform(tips_cat)
ohe_features

Since the resulting matrix is **sparse** – most of its elements are 0 – `sklearn` uses a more efficient representation than a regular `numpy` array. That's no issue, though:

In [ ]:
ohe_features.toarray()

Notice that the column names from `tips_cat` are no longer stored anywhere (remember, `fit` converts the input to a `numpy` array before proceeding).

We can use the `get_feature_names` method on `ohe` to access the names of the one-hot-encoded columns, though:

In [ ]:
ohe.get_feature_names() # x0, x1, x2, and x3 correspond to column names in tips_cat

`ohe` also has an `inverse_transform` method, which takes a one-hot-encoded matrix and returns a categorical matrix.

In [ ]:
ohe.inverse_transform(ohe_features[:10])

## Models in `sklearn`

### Model classes

- `sklearn` model classes (called "estimators") behave like transformers, in that we need to instantiate and `fit` them.
- The difference is that we also need to specify what our "response" or "target" variable is, i.e. what we are trying to predict.
    - Calling `fit` is the same as "training our model".
- There are several models in the [`linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) package; we will start with `LinearRegression`. 

### The `LinearRegression` class

We've seen this a few times in lecture already, but never formally.

In [ ]:
from sklearn.linear_model import LinearRegression

**Important:** From [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression), we have

> LinearRegression fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

In other words, `LinearRegression` minimizes mean squared error by default.

Additionally, by default the `fit_intercept` argument is set to `True`.

In [ ]:
LinearRegression?

### Example: Predicting `'tip'` from `'total_bill'` and `'size'`

In [ ]:
tips.head()

First, we instantiate and fit. By calling `fit`, we are saying "minimize mean squared error and find $w^*$".

In [ ]:
lr = LinearRegression()

# Note that there are two arguments to fit – X and y!
# (It is not necessary to write X= and y=)
lr.fit(X=tips[['total_bill', 'size']], y=tips['tip'])

After fitting, the `predict` method is available. Note that the argument to `predict` can be any 2D array with two columns.

In [ ]:
# Predicted tip from a table of 3 that spends $25 
lr.predict([[25, 3]])

In [ ]:
# Predicted tip from a table of 14 that spends $1000 – probably not accurate!
lr.predict([[1000, 14]])

We can access the intercepts and slopes individually. This model is of the form

$$\text{predicted tip} = w_0^* + w_1^* \cdot \text{total bill} + w_2^* \cdot \text{table size}$$

so we should expect three parameters total.

In [ ]:
lr.intercept_

In [ ]:
lr.coef_

If we want to compute the RMSE of our model, we need to find its predictions on every row in the training data (`tips`).

In [ ]:
all_preds = lr.predict(tips[['total_bill', 'size']])

In [ ]:
np.sqrt(np.mean((all_preds - tips['tip']) ** 2))

It turns out that fit `LinearRegression` objects also have a `score` method:

In [ ]:
lr.score(tips[['total_bill', 'size']], tips['tip'])

That doesn't look like the RMSE... what is it? 🤔

### Aside: $R^2$

- $R^2$, or the **coefficient of determination**, is a measure of the **quality of a linear fit**.
- There are a few equivalent ways of computing it, assuming your model has an intercept term:

$$R^2 = \frac{\text{var}(\text{predicted $y$ values})}{\text{var}(\text{actual $y$ values})}$$

$$R^2 = \left[ \text{correlation}(\text{predicted $y$ values}, \text{actual $y$ values}) \right]^2$$

- In the simple linear regression case, it is the square of the correlation coefficient, $r$.
- **Key idea:** $R^2$ ranges from 0 to 1. **The closer it is to 1, the better the linear fit is.**
- Interpretation: $R^2$ is the **proportion of variance in $y$ that the linear model explains**.

### Calculating $R^2$

Recall, `all_preds` contains the predicted `'tip'` for every data point in `tips`.

In [ ]:
tips.head()

In [ ]:
all_preds[:5]

**Method 1: $R^2 = \frac{\text{var}(\text{predicted $y$ values})}{\text{var}(\text{actual $y$ values})}$**


In [ ]:
np.var(all_preds) / np.var(tips['tip'])

**Method 2:** $R^2 = \left[ \text{correlation}(\text{predicted $y$ values}, \text{actual $y$ values}) \right]^2$

Note: By correlation here, we are referring to $r$.

In [ ]:
(np.corrcoef(all_preds, tips['tip'])) ** 2

**Method 3:** `lr.score`

In [ ]:
lr.score(tips[['total_bill', 'size']], tips['tip'])

All three methods provide the same result!

### `LinearRegression` summary

|Property|Example|Description|
|---|---|---|
|Initialize model parameters| `lr = LinearRegression()` | Create (empty) linear regression model|
|Fit the model to the data | `lr.fit(data, responses)` | Determines regression coefficients|
|Use model for prediction |`lr.predict(newdata)`| Use regression line make predictions|
|Evaluate the model| `lr.score(data, responses)` | Calculate the $R^2$ of the LR model|
|Access model attributes| `lr.coef_` | Access the regression coefficients|

***Note:*** Once `fit`, estimators like `LinearRegression` are just transformers (`predict` <-> `transform`).

## Summary, next time

### Summary

- Quantitative feature transformations allow us to use linear models to model non-linear data.
- Transformers in `sklearn` are used for **feature engineering**, while estimators in `sklearn` are used for **models**.
- A common pattern:
    - Instantiate.
    - `fit`.
    - `transform` / `predict`.
- We like linear models with **low RMSE** and **high $R^2$**!
- **Next time:** Combining transformers and estimators in a single **pipeline**.